<div style="  padding: 10px;text-align: center;" class='row'>
<div style="float:left;width: 10%;" class='column'><a href="https://datos.gov.co/"><img alt="Logo DataSandbox"  src="https://github.com/DataSandbox/Plantilla-Publicacion-Resultados/raw/main/App/logdat.JPG" style="width: 100px;"></a></div>
    <div style="float:left;width: 80%;" class='column'>
        <h1>EXTRACCIÓN DE INFORMACIÓN EN DOCUMENTOS DE PROCESOS DE CONTRATACIÓN PÚBLICA
        </h1> 
    </div>
 <div style="float:left;width: 10%;" class='column'><a href="https://www.colombiacompra.gov.co/" target="_blank"><img class="float-right" src="https://raw.githubusercontent.com/ANCP-CCE-Analitica/datasandbox-extraccion/main/logo_ancp_cce_web.png" style="width: 200px;"></a></div>
    </div>


# Extracción de texto

El ejercicio para implementación utilizo el servicio <span style="color:blue">Cognitive Service - Computer Vision</span> para la extracción de texto en <span style="color:red">Azure DataBricks</span>. El funcionamiento del código exige trabajar en un cluster como el mencionado en conjunto con una <span style="color:green"> Cuenta de almacenamiento de Azure</span>.

In [ ]:
import pandas as pd
spark.conf.set (
  "fs.azure.account.key.dlscontratacionpuproceso.dfs.core.windows.net",
  dbutils.secrets.get (scope = "contratacionpuproceso-key-vault-secrets", key = "StorageAccountKey")
)
file_location = "abfss://demo2@dlscontratacionpuproceso.dfs.core.windows.net/Descarga SECOP I/"
##Lista de archivos
listdirdl=dbutils.fs.ls(file_location+'Descarga SECOP I/')

### Base muestra

df=spark.read.csv(file_location+'Muestra/muestra_SECOP_I.csv',header=True)


## Preparación de los datos para la lectura de texto

Enlistamos los archivos contenidos en la <span style='color:green'>Cuenta de almacenamiento (DataLake)</span> para integrar el PDF con su respectivo proceso:

In [ ]:
Base_archivos=[]
for j,i in enumerate(listdirdl):
  Base_archivos.append([j,i.name.replace('/',''),i.path.replace('abfss://demo2@dlscontratacionpuproceso.dfs.core.windows.net','')])
  
Base_archivos=spark.createDataFrame(Base_archivos,schema=['indice','Name','Path'])

In [ ]:
from pyspark.sql.functions import *
df=df.withColumn("Codigo_contrato", regexp_replace(col("Codigo_contrato"), "'", ""))


### Carga índices


Se arregla el problema de los índices

In [ ]:
indexdf2=spark.read.csv(file_location+'Muestra/index.csv',header=True)


In [ ]:
totaldf=df.join(indexdf2,df.Codigo_contrato==indexdf2.Codigo_contrato,"left")

In [ ]:
totaldf_with_path=totaldf.join(Base_archivos,totaldf.Indice==Base_archivos.Name,'left')


In [ ]:
pdtotaldf_with_path=totaldf_with_path.toPandas()

In [ ]:
pdtotaldf_with_path=pdtotaldf_with_path[~pdtotaldf_with_path.Indice.isna()]

Se renombran las columnas:

In [ ]:
columns2=['UID', 'Anno_Cargue_SECOP', 'Anno_Firma_del Contrato', 'Nivel Entidad',
       'Orden Entidad', 'Nombre de la Entidad', 'NIT de la Entidad',
       'Código de la Entidad', 'ID Tipo de Proceso', 'Tipo de Proceso',
       'Estado del Proceso', 'Causal de Otras Formas de Contratacion Directa',
       'ID Regimen de Contratacion', 'Regimen de Contratacion',
       'ID Objeto a Contratar', 'Objeto a Contratar',
       'Detalle del Objeto a Contratar', 'Tipo de Contrato',
       'Municipio Obtencion', 'Municipio Entrega', 'Municipios Ejecucion',
       'Fecha de Cargue en el SECOP', 'Numero de Constancia',
       'Numero de Proceso', 'Numero del Contrato', 'Cuantia Proceso',
       'ID Grupo', 'Nombre Grupo', 'ID Familia', 'Nombre Familia', 'ID Clase',
       'Nombre Clase', 'ID Ajudicacion', 'Tipo Identifi del Contratista',
       'Identificacion del Contratista', 'Nom Raz Social Contratista',
       'Dpto y Muni Contratista', 'Tipo Doc Representante Legal',
       'Identific del Represen Legal', 'Nombre del Represen Legal',
       'Fecha de Firma del Contrato', 'Fecha Ini Ejec Contrato',
       'Plazo de Ejec del Contrato', 'Rango de Ejec del Contrato',
       'Tiempo Adiciones en Dias', 'Tiempo Adiciones en Meses',
       'Fecha Fin Ejec Contrato', 'Compromiso Presupuestal',
       'Cuantia Contrato', 'Valor Total de Adiciones',
       'Valor Contrato con Adiciones', 'Objeto del Contrato a la Firma',
       'ID Origen de los Recursos', 'Origen de los Recursos', 'Codigo BPIN',
       'Proponentes Seleccionados', 'Calificacion Definitiva',
       'ID Sub Unidad Ejecutora', 'Nombre Sub Unidad Ejecutora',
       'Ruta Proceso en SECOP I', 'Moneda', 'EsPostConflicto',
       'Marcacion Adiciones', 'Posicion Rubro', 'Nombre Rubro', 'Valor Rubro',
       'Sexo RepLegal Entidad', 'Pilar Acuerdo Paz', 'Punto Acuerdo Paz',
       'Municipio Entidad', 'Departamento Entidad', 'Ultima Actualizacion',
       'Fecha Liquidacion', 'Codigo_contrato_2', 'Codigo_contrato', 'Cantidad',
       'Indice', 'indice_2', 'Name', 'Path']

In [ ]:
columns2=[i.replace(" ","_") for i in columns2]
pdtotaldf_with_path.columns=columns2

In [ ]:
totaldf_with_path=spark.createDataFrame(pdtotaldf_with_path)


In [ ]:
#totaldf_with_path.write.parquet(file_location+'/Descarga SECOP I/MUESTRA/totaldf_path_2.parquet')

In [ ]:
file_location='abfss://demo2@dlscontratacionpuproceso.dfs.core.windows.net'

In [ ]:
name_file=dbutils.fs.ls(file_location+'/Descarga SECOP I/Descarga SECOP I/16-12-4904596-4497785/')[0].name


## Uso de Computer Vision

Empezamos a desplegar <span style='color:blue'> Computer Vision </span> para poder leer el texto de los documentos:

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes

stg_conn_str = "DefaultEndpointsProtocol=https;AccountName=dlscontratacionpuproceso;AccountKey=" + dbutils.secrets.get(scope = "contratacionpuproceso-key-vault-secrets", key = "StorageAccountKey") + ";EndpointSuffix=core.windows.net"


cog_key = dbutils.secrets.get(scope = "contratacionpuproceso-key-vault-secrets", key = "ComputerVisionKey")
cog_endpoint = 'https://cog-computervision-contpubli-procesos.cognitiveservices.azure.com/'




client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

In [ ]:
def pdf_text(path):
    blob_client = BlobClient.from_connection_string(conn_str=stg_conn_str,container_name="demo2",blob_name=path)
    # Images PDF with text
    with io.BytesIO() as blob_stream:
      blob = blob_client.download_blob()
      blob.download_to_stream(blob_stream)
      blob_stream.seek(0)
      response = client.read_in_stream(blob_stream, raw=True) 

    # Get ID from returned headers
    operation_location = response.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    # SDK call that gets what is read
    while True:
        result = client.get_read_result(operation_id)
        if result.status.lower() not in ['notstarted', 'running']:
            break
        #print ('Waiting for result...')
        time.sleep(10)
    return result

In [ ]:
error=[]
for j in range(208):
  List_corpus=[]
  List_corpus_lines=[]
  List_path=[]  
  corpus=dict()
  for n,i in enumerate(pdtotaldf_with_path.Path[j*100:(j+1)*100]): 
    Texto=[]
    try:
      name_file=dbutils.fs.ls(file_location+i)[0].name
      result=pdf_text(i+name_file)  
      if result.status == OperationStatusCodes.succeeded:
        for readResult in result.analyze_result.read_results:
          for line in readResult.lines:
            Texto.append(line.text)
        if n==0:
          print(j+n)
      else:
        error.append((j+n,i))
    except:
      error.append((j+n,i))
      continue
    List_corpus_lines.append(Texto)
    List_corpus.append('\n'.join(Texto))
    List_path.append(i)
  corpus['Path']=List_path
  corpus['Text_List']=List_corpus_lines
  corpus['Text']=List_corpus
  dfcorpus1=pd.DataFrame(corpus)
  corpus1 = spark.createDataFrame(dfcorpus1)
  corpus1.write.parquet(file_location+'/Descarga SECOP I/Muestra/corpus'+str(j)+'.parquet',mode='overwrite')
  

En este vector visualizamos los documentos que no se pudieron leer, este ejercicio permite que <span style='color:blue'> Computer Vision </span> identifique que archiivos no pueden leerse por su calidad deficiente. Así mismo se puede apreciar que se generan corpus por cada 100 contratos leidos.

In [ ]:
error

In [ ]:
df=spark.read.parquet(file_location+'/Descarga SECOP I/Muestra/corpus1.parquet',mode='overwrite')
display(df)


In [ ]:
dbutils.fs.ls(file_location+'/Descarga SECOP I/Muestra/')

In [ ]:
Base_archivos.write.parquet(file_location+'/Descarga SECOP I/Muestra/muestra_SECOP_I.parquet')